<a href="https://colab.research.google.com/github/buseerkiraz/CMPE58T_AP/blob/main/CMPE58T_Application_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
This project is for CMPE58T (Advanced Natural Language Processing) Master's course in Boğaziçi
University. It investigates methods to improve Turkish-to-English neural machine translation
(NMT) by incorporating various types of linguistic and stylistic guidance into a baseline
MarianMT model.

The project is structured around several enhancements to a pre-trained baseline model
(Helsinki-NLP/opus-mt-tr-en). First, a simple fine-tuning is performed on the WMT17 parallel
corpus. Then, a morphological variant is developed by integrating rich morphological analyses
(using the Sak et al. 2007 morphological parser), which are reformatted into a ConcatMorph
structure to explicitly encode grammatical information in the input.

A third direction involves training two separate GPT-2 based language models (LMs) — one on
naturally-authored English sentences (from NewsCrawl) and another on machine-translated English
sentences (from the GTNC corpus). These LMs are then used to compute naturalness scores for
each sentence in the training set, allowing the model to tag them with either `<nat>` or
`<trans>`. These tags are prepended to source sentences in order to fine-tune a NaturalMT
model capable of producing more fluent and human-like translations.

The models — Baseline, FinetunedMT, MorphMT, and NaturalMT — are compared both qualitatively
(via direct translation examples) and quantitatively (using BLEU, chrF++, and BERTScore metrics).
All training and evaluation outputs, including translations and metrics, are stored in Google
Drive for reproducibility and analysis.

Ultimately, the goal of this project is to explore how morphological awareness and stylistic
fluency cues (naturalness tagging) can enhance the quality of machine translation systems
for morphologically rich, low-resource languages like Turkish.
"""


In [ ]:
# Install a specific version of the Hugging Face Transformers library
! pip install transformers==4.51.3 --quiet

# Install additional libraries:
# - datasets: for accessing and processing NLP datasets
# - sacrebleu: for computing BLEU scores in a standardized way
# - sentencepiece: for tokenization, commonly used in multilingual models
# - accelerate: for optimized training and inference across devices
# - evaluate: for evaluating model predictions
! pip install datasets sacrebleu sentencepiece accelerate evaluate

# Import the `load_dataset` function to load datasets from the Hugging Face Hub
from datasets import load_dataset

# Import the Dataset class to create or manipulate datasets in Hugging Face format
from datasets import Dataset

# Import PyTorch for model loading and tensor manipulation
import torch

# Import the `evaluate` module for evaluation metrics.
import evaluate

# Install the `bert_score` library for semantic evaluation using BERT embeddings
! pip install bert_score

# Import standard libraries for file handling and date-time processing
import os
import json
from datetime import datetime


In [ ]:
# BASELINE MODEL

# Load the pre-trained MarianMT model and tokenizer for Turkish-to-English translation
from transformers import MarianMTModel, MarianTokenizer

baseline_model_name = "Helsinki-NLP/opus-mt-tr-en"
baseline_model = MarianMTModel.from_pretrained(baseline_model_name)
baseline_tokenizer = MarianTokenizer.from_pretrained(baseline_model_name)


# Function to load parallel data (source and target files)
# Returns a HuggingFace Dataset object with aligned sentence pairs
def load_parallel_data(src_path, tgt_path, src_lang="tr", tgt_lang="en"):
    with open(src_path, encoding="utf-8") as f_src, open(tgt_path, encoding="utf-8") as f_tgt:
        src_lines = [line.strip() for line in f_src]
        tgt_lines = [line.strip() for line in f_tgt]
    data = [{"translation": {src_lang: s, tgt_lang: t}} for s, t in zip(src_lines, tgt_lines)]
    return Dataset.from_list(data)

# Load WMT17 test set consisting of Turkish-English sentence pairs
test_dataset = load_parallel_data("test.tr", "test.en")


# Set device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
baseline_model.to(device)  # Move model to the selected device
baseline_model.eval()      # Set model to evaluation mode (no dropout etc.)

# Load the evaluation metric for BLEU
metric = evaluate.load("sacrebleu")

# Extract source (TR) and reference (EN) sentences from the dataset
sources = [ex["translation"]["tr"] for ex in test_dataset]
references = [ex["translation"]["en"] for ex in test_dataset]

# Placeholder list to store model predictions
baseline_predictions = []
batch_size = 16  # Define batch size for inference

# Perform batch inference using beam search decoding
for i in range(0, len(sources), batch_size):
    batch_sources = sources[i:i+batch_size]
    batch_refs = references[i:i+batch_size]

    # Tokenize batch of source sentences
    inputs = baseline_tokenizer(batch_sources, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move input tensors to device

    # Generate translations without updating gradients
    with torch.no_grad():
        output_ids = baseline_model.generate(**inputs, max_length=128, num_beams=4)  # Beam search decoding
        batch_preds = baseline_tokenizer.batch_decode(output_ids, skip_special_tokens=True)  # Convert token IDs to strings
        baseline_predictions.extend(batch_preds)

# Evaluate model predictions with BLEU, BERTScore, and ChrF++
bleu = evaluate.load("sacrebleu").compute(predictions=baseline_predictions, references=[[ref] for ref in references])
bertscore_result = evaluate.load("bertscore").compute(predictions=baseline_predictions, references=references, lang="en")
chrf_result = evaluate.load("chrf").compute(predictions=baseline_predictions, references=references)


# Mount Google Drive to save experiment outputs
from google.colab import drive
drive.mount('/content/drive')

# Define a folder for saving baseline model outputs
baseline_dir = "/content/drive/MyDrive/mt_experiments/baseline"
os.makedirs(baseline_dir, exist_ok=True)  # Create directory if it doesn't exist


# Save translations as a JSONL file (line-delimited JSON for each example)
translations_jsonl_path = os.path.join(baseline_dir, "baseline-opus-mt-tr-en.translations.jsonl")
with open(translations_jsonl_path, "w", encoding="utf-8") as f:
    for src, pred, ref in zip(sources, baseline_predictions, references):
        line = {"source": src, "prediction": pred, "reference": ref}
        f.write(json.dumps(line, ensure_ascii=False) + "\n")


# Save evaluation metrics (BLEU, BERTScore, ChrF++) in a JSON file
eval_result = {
    "timestamp": datetime.now().isoformat(),
    "BLEU": bleu["score"],
    "BERTScore": {
        "precision": sum(bertscore_result["precision"]) / len(bertscore_result["precision"]),
        "recall": sum(bertscore_result["recall"]) / len(bertscore_result["recall"]),
        "f1": sum(bertscore_result["f1"]) / len(bertscore_result["f1"]),
    },
    "ChrF++": chrf_result["score"]
}

eval_json_path = os.path.join(baseline_dir, "baseline_evaluation_results.json")
with open(eval_json_path, "w", encoding="utf-8") as f:
    json.dump(eval_result, f, indent=4)  # Save metrics in a readable format


# Save predictions in a readable TXT file (source, prediction, reference)
txt_path = os.path.join(baseline_dir, "baseline_test_predictions.txt")
with open(txt_path, "w", encoding="utf-8") as f:
    for i in range(len(sources)):
        f.write(f"{i+1}.\n")
        f.write(f"SOURCE (TR):    {sources[i]}\n")
        f.write(f"BASELINE PRED:  {baseline_predictions[i]}\n")
        f.write(f"REFERENCE (EN): {references[i]}\n")
        f.write("-" * 60 + "\n")

# Confirm that outputs have been saved
print("baselineMT outputs saved to:", baseline_dir)


In [ ]:
# FINETUNED MODEL

from sklearn.model_selection import train_test_split

# Paths to original WMT parallel corpus
en_path = "WMT-News.en-tr.en"
tr_path = "WMT-News.en-tr.tr"

# Read English and Turkish lines from files
with open(en_path, 'r', encoding='utf-8') as f_en, open(tr_path, 'r', encoding='utf-8') as f_tr:
    en_lines = [line.strip() for line in f_en]
    tr_lines = [line.strip() for line in f_tr]

# Ensure both files have the same number of lines
assert len(en_lines) == len(tr_lines), "Line count mismatch..."

# Split the data into 80% train, 10% validation, 10% test
train_en, temp_en, train_tr, temp_tr = train_test_split(en_lines, tr_lines, test_size=0.2, random_state=42)
val_en, test_en, val_tr, test_tr = train_test_split(temp_en, temp_tr, test_size=0.5, random_state=42)

# Save split data to files
def save_pairs(prefix, en, tr):
    with open(f"{prefix}.en", "w", encoding="utf-8") as f_en, open(f"{prefix}.tr", "w", encoding="utf-8") as f_tr:
        for e, t in zip(en, tr):
            f_en.write(e + "\n")
            f_tr.write(t + "\n")

save_pairs("train", train_en, train_tr)
save_pairs("val", val_en, val_tr)
save_pairs("test", test_en, test_tr)

# Load dataset using HuggingFace's text loader (not yet tokenized or structured)
from datasets import load_dataset
dataset = load_dataset("text", data_files={"en": "train.en", "tr": "train.tr"})

# OPTIONAL DEBUG: Show N lines of raw data
with open(en_path, 'r', encoding='utf-8') as f_en, open(tr_path, 'r', encoding='utf-8') as f_tr:
    for i in range(num_lines_to_show):  # variable must be defined elsewhere
        en_line = f_en.readline().strip()
        tr_line = f_tr.readline().strip()
        print(f"{i+1}. EN: {en_line}")
        print(f"   TR: {tr_line}\n")

# Utility to load parallel data into a HuggingFace-compatible dataset
def load_parallel_data(src_path, tgt_path, src_lang="tr", tgt_lang="en"):
    with open(src_path, encoding="utf-8") as f_src, open(tgt_path, encoding="utf-8") as f_tgt:
        src_lines = [line.strip() for line in f_src]
        tgt_lines = [line.strip() for line in f_tgt]
    data = [{"translation": {src_lang: s, tgt_lang: t}} for s, t in zip(src_lines, tgt_lines)]
    return Dataset.from_list(data)

# Load preprocessed training and test datasets
train_dataset = load_parallel_data("train.tr", "train.en")
test_dataset = load_parallel_data("test.tr", "test.en")

# Load pretrained MarianMT model and tokenizer for Turkish-English
model_name = "Helsinki-NLP/opus-mt-tr-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Preprocessing function for tokenizing input-output sentence pairs
def preprocess_function(examples):
    inputs = [ex["tr"] for ex in examples["translation"]]
    targets = [ex["en"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize training and validation data
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)  # val_dataset needs to be defined beforehand

# Import training classes from Transformers
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Define training arguments for fine-tuning
training_args = Seq2SeqTrainingArguments(
    output_dir="./opus-mt-tr-en-finetuned",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",  # disables WandB or TensorBoard logging
)

# Define the Seq2SeqTrainer for training and evaluation
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)

# Start fine-tuning
trainer.train()

# Save the fine-tuned model and tokenizer
trainer.save_model("fine-tuned-opus-mt-tr-en")
tokenizer.save_pretrained("fine-tuned-opus-mt-tr-en")

# Mount Google Drive to save the fine-tuned model there as well
from google.colab import drive
drive.mount('/content/drive')

# Save model and tokenizer to Drive
trainer.save_model("/content/drive/MyDrive/fine-tuned-opus-mt-tr-en")
tokenizer.save_pretrained("/content/drive/MyDrive/fine-tuned-opus-mt-tr-en")

# Reload test dataset for final evaluation
def load_parallel_data(src_path, tgt_path, src_lang="tr", tgt_lang="en"):
    with open(src_path, encoding="utf-8") as f_src, open(tgt_path, encoding="utf-8") as f_tgt:
        src_lines = [line.strip() for line in f_src]
        tgt_lines = [line.strip() for line in f_tgt]
    data = [{"translation": {src_lang: s, tgt_lang: t}} for s, t in zip(src_lines, tgt_lines)]
    return Dataset.from_list(data)

test_dataset = load_parallel_data("test.tr", "test.en")

# Setup for evaluation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Load evaluation metrics
metric = evaluate.load("sacrebleu")
bertscore = evaluate.load("bertscore")
chrf = evaluate.load("chrf")

# Extract source and reference sentences
sources = [ex["translation"]["tr"] for ex in test_dataset]
references = [ex["translation"]["en"] for ex in test_dataset]

# Generate translations in batches
predictions = []
batch_size = 16

for i in range(0, len(sources), batch_size):
    batch_sources = sources[i:i+batch_size]
    batch_refs = references[i:i+batch_size]

    inputs = tokenizer(batch_sources, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        output_ids = model.generate(**inputs, max_length=128, num_beams=4)
        batch_preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        predictions.extend(batch_preds)

# Compute and print BLEU score
bleu = metric.compute(predictions=predictions, references=[[ref] for ref in references])
print(f"\nTest BLEU score: {bleu['score']:.2f}")

# Compute BERTScore (semantic similarity)
bertscore_result = bertscore.compute(predictions=predictions, references=references, lang="en")

# Compute ChrF++ (character n-gram F-score)
chrf_result = chrf.compute(predictions=predictions, references=references)
print(f"\nChrF++ score: {chrf_result['score']:.2f}")

# Save all evaluation metrics to disk
def save_eval_result(model_name, bleu, bertscore_result, chrf_result, save_path="evaluation_results.json"):
    new_entry = {
        "timestamp": datetime.now().isoformat(),
        "BLEU": bleu["score"],
        "BERTScore": {
            "precision": sum(bertscore_result["precision"]) / len(bertscore_result["precision"]),
            "recall": sum(bertscore_result["recall"]) / len(bertscore_result["recall"]),
            "f1": sum(bertscore_result["f1"]) / len(bertscore_result["f1"]),
        },
        "ChrF++": chrf_result["score"]
    }

    # Append to existing file or create a new one
    if os.path.exists(save_path):
        with open(save_path, "r", encoding="utf-8") as f:
            all_results = json.load(f)
    else:
        all_results = {}

    all_results[model_name] = new_entry

    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(all_results, f, indent=4)

    print(f"Evaluation saved to '{model_name}'")

# Save evaluation scores to a JSON file
save_eval_result(
    model_name="fine-tuned-opus-mt-tr-en",
    bleu=bleu,
    bertscore_result=bertscore_result,
    chrf_result=chrf_result
)

# Save translations (source, reference, prediction) as JSONL
def save_translations_jsonl(model_name, sources, references, predictions, filename=None):
    if filename is None:
        filename = f"{model_name}.translations.jsonl"

    with open(filename, "w", encoding="utf-8") as f:
        for src, ref, pred in zip(sources, references, predictions):
            json_line = {
                "source": src,
                "reference": ref,
                "prediction": pred
            }
            f.write(json.dumps(json_line, ensure_ascii=False) + "\n")

    print(f"Saved translations to {filename}")

save_translations_jsonl("fine-tuned-opus-mt-tr-en", sources, references, predictions)

# Save evaluation scores again (redundant, but safe if done in separate runtime)
save_eval_result(
    model_name="fine-tuned-opus-mt-tr-en",
    bleu=bleu,
    bertscore_result=bertscore_result,
    chrf_result=chrf_result
)

In [ ]:
# MORPH MT

# Download and unzip morphological parser files (Sak et al. (2007))
!gdown --id 1rja1ejQPACYeRwGwb1TuwZJeUpiAfM4a
!unzip Parser.zip

# Install Python 2, which is required by the parser
!apt-get install python2

from concurrent.futures import ProcessPoolExecutor
import os
import re

# Define a simple tokenizer using regex to separate punctuation
def basic_tokenize(text):
    # Surround punctuation with spaces
    text = re.sub(r"([.,!?()\":;])", r" \1 ", text)
    # Collapse multiple spaces into one
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip()

# Apply the basic tokenizer to each line of test.tr and write to test.txt
with open("test.tr", encoding="utf-8") as fin, open("test.txt", "w", encoding="utf-8") as fout:
    for line in fin:
        if line.strip():
            fout.write(basic_tokenize(line.strip()) + "\n")

# Function to run the morphological parser script in Python 2
def run_script(filename):
    os.system(f'python2.7 parse_corpus.py "{filename}" > "morph_{filename}"')

# List of files to parse (only test.txt here)
filenames = ["test.txt",]

# Run the parser in parallel (for larger batches)
with ProcessPoolExecutor() as executor:
    executor.map(run_script, filenames)

# Disambiguate morphological analyses using provided Perl script and model
!perl md.pl -disamb model.m "morph_test.txt" "disamb_test.txt"

# Compress disambiguated output for backup
!zip morph_files.zip disamb_test.txt*

# Parse the output of the morphological disambiguator
def parse_first_analysis(line):
    """
    Extracts the first morphological analysis in word<lemma+TAG1+TAG2+...> format.
    Returns 'word<unknown>' if analysis is missing or unknown.
    """
    if line.startswith('<S') or line.startswith('</S') or not line.strip():
        return None

    parts = line.strip().split()
    word = parts[0]

    if '[Unknown]' in line:
        return f"{word}<unknown>"

    for token in parts[1:]:
        if '[' not in token:
            continue
        analyses = re.findall(r"([^+\[\]]*)\[([^\[\]]+)\]", token)
        if analyses:
            lemma = analyses[0][0]
            tags = [t for (_, t) in analyses]
            tag_str = '+'.join(tags)
            return f"{word}<{lemma}+{tag_str}>"

    return f"{word}<unknown>"

# Process full file of disambiguated morphological analyses
def process_file(file_path, output_path):
    sentences = []
    sentence = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line.startswith('<S'):
                sentence = []
            elif line.startswith('</S'):
                if sentence:
                    sentences.append(' '.join(sentence))
            else:
                parsed = parse_first_analysis(line)
                if parsed:
                    sentence.append(parsed)
    with open(output_path, 'w', encoding='utf-8') as out:
        for s in sentences:
            out.write(s + '\n')

# Convert morphological parser output into sentence format
process_file("disamb_test.txt", "final_test.tr")

from pathlib import Path

# Convert parsed morphological forms into ConcatMorph format: gidiyorum_gitVerbProgPres1Sg
def convert_to_concatmorph_format(tr_input_path, en_input_path, tr_output_path):
    tr_lines = Path(tr_input_path).read_text(encoding='utf-8').splitlines()
    en_lines = Path(en_input_path).read_text(encoding='utf-8').splitlines()

    converted_lines = []

    for line in tr_lines:
        words = line.strip().split()
        converted = []

        for word in words:
            if '<' in word and '>' in word:
                try:
                    base, morph = word.split('<', 1)
                    morph = morph.rstrip('>')
                    if '+Punc' in morph:
                        converted_word = base  # exclude punctuation tags
                    elif morph == "unknown":
                        converted_word = base
                    else:
                        morph_concat = "_" + morph.replace("+", "")
                        converted_word = base + morph_concat
                except ValueError:
                    converted_word = word
            else:
                converted_word = word

            converted.append(converted_word)

        converted_lines.append(" ".join(converted))

    Path(tr_output_path).write_text("\n".join(converted_lines), encoding='utf-8')
    print(f"Converted {len(converted_lines)} lines to ConcatMorph format.")
    print("Turkish and English files are aligned.")

# Run ConcatMorph conversion
convert_to_concatmorph_format("final_test.tr", "test.en", "test.concatmorph.tr")

# Load parallel dataset into Hugging Face Dataset object
def load_parallel_data(src_path, tgt_path, src_lang="tr", tgt_lang="en"):
    with open(src_path, encoding="utf-8") as f_src, open(tgt_path, encoding="utf-8") as f_tgt:
        src_lines = [line.strip() for line in f_src]
        tgt_lines = [line.strip() for line in f_tgt]
    data = [{"translation": {src_lang: s, tgt_lang: t}} for s, t in zip(src_lines, tgt_lines)]
    return Dataset.from_list(data)

# Load train/test datasets in ConcatMorph format
train_dataset = load_parallel_data("train.concatmorph.tr", "train.en")
# val_dataset = load_parallel_data("final_val.tr", "val.en")  # Optional
test_dataset = load_parallel_data("test.concatmorph.tr", "test.en")

# Load pretrained model and tokenizer
model_name = "Helsinki-NLP/opus-mt-tr-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Preprocess the tokenized dataset for Seq2Seq training
def preprocess_function(examples):
    inputs = [ex["tr"] for ex in examples["translation"]]
    targets = [ex["en"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize the datasets
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

# Define training arguments for the morph-aware model
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./opus-mt-tr-en-finetuned",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
)

# Train with Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
)

trainer.train()

# Save the trained morph model locally
trainer.save_model("morph-opus-mt2-tr-en")
tokenizer.save_pretrained("morph-opus-mt2-tr-en")

# Mount Google Drive to store trained model
from google.colab import drive
drive.mount('/content/drive')

# Save the model to Google Drive
trainer.save_model("/content/drive/MyDrive/morph-opus-mt2-tr-en")
tokenizer.save_pretrained("/content/drive/MyDrive/morph-opus-mt2-tr-en")

# Reload test set
def load_parallel_data(src_path, tgt_path, src_lang="tr", tgt_lang="en"):
    with open(src_path, encoding="utf-8") as f_src, open(tgt_path, encoding="utf-8") as f_tgt:
        src_lines = [line.strip() for line in f_src]
        tgt_lines = [line.strip() for line in f_tgt]
    data = [{"translation": {src_lang: s, tgt_lang: t}} for s, t in zip(src_lines, tgt_lines)]
    return Dataset.from_list(data)

test_dataset = load_parallel_data("test.concatmorph.tr", "test.en")

# Prepare model and device for evaluation
sources = [ex["translation"]["tr"] for ex in test_dataset]
references = [ex["translation"]["en"] for ex in test_dataset]

morph_model_path = "morph-opus-mt2-tr-en"
morph_tokenizer = MarianTokenizer.from_pretrained(morph_model_path)
morph_model = MarianMTModel.from_pretrained(morph_model_path).to("cuda" if torch.cuda.is_available() else "cpu")
morph_model.eval()

morph_predictions = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 16

# Run inference on morph-aware model
for i in range(0, len(sources), batch_size):
    batch_sources = sources[i:i+batch_size]

    inputs = morph_tokenizer(batch_sources, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        output_ids = morph_model.generate(**inputs, max_length=128, num_beams=4)
        batch_preds = morph_tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        morph_predictions.extend(batch_preds)

# Evaluate morph model
bleu = evaluate.load("sacrebleu").compute(predictions=morph_predictions, references=[[ref] for ref in references])
bertscore_result = evaluate.load("bertscore").compute(predictions=morph_predictions, references=references, lang="en")
chrf_result = evaluate.load("chrf").compute(predictions=morph_predictions, references=references)

# Save evaluation results and predictions to Drive
morph_dir = "/content/drive/MyDrive/mt_experiments/morph"
os.makedirs(morph_dir, exist_ok=True)

# Save translations as JSONL
translations_jsonl_path = os.path.join(morph_dir, "morph-opus-mt2-tr-en.translations.jsonl")
with open(translations_jsonl_path, "w", encoding="utf-8") as f:
    for src, pred, ref in zip(sources, morph_predictions, references):
        line = {"source": src, "prediction": pred, "reference": ref}
        f.write(json.dumps(line, ensure_ascii=False) + "\n")

# Save evaluation metrics
eval_result = {
    "timestamp": datetime.now().isoformat(),
    "BLEU": bleu["score"],
    "BERTScore": {
        "precision": sum(bertscore_result["precision"]) / len(bertscore_result["precision"]),
        "recall": sum(bertscore_result["recall"]) / len(bertscore_result["recall"]),
        "f1": sum(bertscore_result["f1"]) / len(bertscore_result["f1"]),
    },
    "ChrF++": chrf_result["score"]
}

eval_json_path = os.path.join(morph_dir, "morph_evaluation_results.json")
with open(eval_json_path, "w", encoding="utf-8") as f:
    json.dump(eval_result, f, indent=4)

# Save readable prediction log
txt_path = os.path.join(morph_dir, "morph_test_predictions.txt")
with open(txt_path, "w", encoding="utf-8") as f:
    for i in range(len(sources)):
        f.write(f"{i+1}.\n")
        f.write(f"SOURCE (TR):    {sources[i]}\n")
        f.write(f"MORPH PRED:     {morph_predictions[i]}\n")
        f.write(f"REFERENCE (EN): {references[i]}\n")
        f.write("-" * 80 + "\n")

print("All morph outputs saved to", morph_dir)

In [ ]:
# NATURALMT: Train two LMs (natural & translationese), tag data, fine-tune NMT

from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
import sys

# Download and preprocess natural English monolingual corpus (NewsCrawl)
!wget http://data.statmt.org/news-crawl/en/news.2020.en.shuffled.deduped.gz
!gunzip news.2020.en.shuffled.deduped.gz
!mkdir -p lm_data/natural

# Filter lines by length and save 20K for training the natural LM
!cat news.2020.en.shuffled.deduped \
  | awk 'length > 20 && length < 200' \
  | head -n 20000 > lm_data/natural/english.txt

# Clone GTNC repository which contains Google Translations from NewsCrawl
!git clone https://github.com/damiaanr/gtnc

# Prepare translationese LM data by sampling lines from GTNC outputs
import os
from pathlib import Path
import random

GTNC_TRANSLATED_DIR = Path("gtnc/output/translated")
OUTPUT_FILE = Path("lm_data/translationese/translationese_english.txt")
TARGET_LINE_COUNT = 20000
LANGUAGES = ["tr", "de", "zh"]  # languages used to generate translationese

def collect_selected_trg_files(directory, selected_langs):
    files = []
    for lang in selected_langs:
        file_path = directory / f"{lang}.trg"
        if file_path.exists():
            files.append(file_path)
        else:
            print(f"File not found: {file_path}")
    return files

def sample_sentences(files, max_lines=20000):
    all_lines = []
    for f in files:
        with f.open("r", encoding="utf-8") as file:
            lines = [line.strip() for line in file if line.strip()]
            all_lines.extend(lines)
    random.shuffle(all_lines)
    return all_lines[:max_lines]

def main():
    os.makedirs(OUTPUT_FILE.parent, exist_ok=True)
    trg_files = collect_selected_trg_files(GTNC_TRANSLATED_DIR, LANGUAGES)
    print(f"Using files: {[f.name for f in trg_files]}")
    sampled = sample_sentences(trg_files, TARGET_LINE_COUNT)
    print(f"Writing {len(sampled)} lines to {OUTPUT_FILE}...")
    with OUTPUT_FILE.open("w", encoding="utf-8") as out:
        for line in sampled:
            out.write(line + "\n")
    print("done")

if __name__ == "__main__":
    main()

# Disable WandB logging
os.environ["WANDB_DISABLED"] = "true"

# Setup and train GPT-2 based language model (either natural or translationese)
from transformers import *
from pathlib import Path

DATA_FILE = "natural_english.txt"  # set this to translationese_english.txt to train transLM
OUTPUT_DIR = "lm_nat_model"
EPOCHS = 3

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Load dataset for LM fine-tuning
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=DATA_FILE,
    block_size=128,
    overwrite_cache=True,
)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Define training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=2,
    eval_strategy="no",
    logging_dir=f"{OUTPUT_DIR}/logs",
    save_total_limit=2,
    save_steps=500,
    logging_steps=10,
)

# Train language model
trainer = Trainer(model=model, args=training_args, data_collator=data_collator, train_dataset=dataset)
trainer.train()

# Save the model
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"Model trained and saved in {OUTPUT_DIR}")

# Save trained LM to Google Drive
from google.colab import drive
import shutil

drive.mount('/content/drive')
models = {"lm_nat_model": "/content/drive/MyDrive/NMT_LMs/LM_Natural"}
for local_path, drive_path in models.items():
    shutil.copytree(local_path, drive_path, dirs_exist_ok=True)

print("Models saved to Google Drive")

# Load both LMs for scoring sentences
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from pathlib import Path
from tqdm import tqdm

# Paths and settings
NAT_MODEL_DIR = "lm_nat_model"
TRANS_MODEL_DIR = "lm_trans_model"
SOURCE_TR = "train.tr"
TARGET_EN = "train.en"
OUTPUT_TAGGED_TR = "tagged.tr"
OUTPUT_TAGGED_EN = "tagged.en"
THRESHOLD = 0.0

# Load models and tokenizers
tok_nat = GPT2Tokenizer.from_pretrained(NAT_MODEL_DIR)
model_nat = GPT2LMHeadModel.from_pretrained(NAT_MODEL_DIR).eval()

tok_trans = GPT2Tokenizer.from_pretrained(TRANS_MODEL_DIR)
model_trans = GPT2LMHeadModel.from_pretrained(TRANS_MODEL_DIR).eval()

def get_score(sentence, model, tokenizer):
    inputs = tokenizer.encode(sentence, return_tensors="pt", truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(inputs, labels=inputs)
        return -outputs.loss.item()  # log-probability

# Load training pairs
with open(SOURCE_TR, encoding="utf-8") as f_tr, open(TARGET_EN, encoding="utf-8") as f_en:
    src_lines = [line.strip() for line in f_tr]
    tgt_lines = [line.strip() for line in f_en]

assert len(src_lines) == len(tgt_lines), "Mismatch between TR and EN lines!"

# Tag sentences based on naturalness score comparison
tagged_src = []
tagged_tgt = []
for src, tgt in tqdm(zip(src_lines, tgt_lines), total=len(src_lines)):
    p_nat = get_score(tgt, model_nat, tok_nat)
    p_trans = get_score(tgt, model_trans, tok_trans)
    tag = "<nat>" if (p_nat - p_trans) > THRESHOLD else "<trans>"
    tagged_src.append(f"{tag} {src}")
    tagged_tgt.append(tgt)

# Save tagged training files
Path("tagged.tr").write_text("\n".join(tagged_src), encoding="utf-8")
Path("tagged.en").write_text("\n".join(tagged_tgt), encoding="utf-8")

# Display some examples
for i, line in enumerate(tagged_src[:20]):
    print(f"{i+1:>4}: {line}")

# Count tag distribution
from collections import Counter
tags = [line.split()[0] for line in tagged_src if line.strip().startswith("<")]
counts = Counter(tags)
print("Tag counts in tagged.tr:")
print(f"  <nat>   : {counts.get('<nat>', 0)}")
print(f"  <trans> : {counts.get('<trans>', 0)}")
print(f"  Total   : {sum(counts.values())}")

# Fine-tune NMT model on tagged dataset
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset

src_lines = Path("tagged.tr").read_text(encoding="utf-8").splitlines()
tgt_lines = Path("tagged.en").read_text(encoding="utf-8").splitlines()
assert len(src_lines) == len(tgt_lines)

MODEL_NAME = "Helsinki-NLP/opus-mt-tr-en"
OUTPUT_DIR = "natural_nmt_model"
EPOCHS = 3

tokenizer = MarianTokenizer.from_pretrained(MODEL_NAME)
model = MarianMTModel.from_pretrained(MODEL_NAME)

# Add <nat> and <trans> tokens to tokenizer
special_tokens = ["<nat>", "<trans>"]
tokenizer.add_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))

# Convert to HF Dataset
raw_data = {"translation": [{"tr": tr, "en": en} for tr, en in zip(src_lines, tgt_lines)]}
dataset = Dataset.from_list(raw_data["translation"])

def preprocess(example):
    model_inputs = tokenizer(example["tr"], truncation=True, max_length=128, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example["en"], truncation=True, max_length=128, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess)

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="no",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=500,
    logging_dir=f"{OUTPUT_DIR}/logs",
    logging_steps=10,
)

# Train
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)
trainer.train()

# Save to disk
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"NMT model trained and saved to {OUTPUT_DIR}")

# Save to Google Drive
drive.mount('/content/drive')
shutil.copytree("natural_nmt_model", "/content/drive/MyDrive/NMT_LMs/Natural_NMT_Model", dirs_exist_ok=True)

# Inference on new sentences
def translate_tr_sentences(sentences, style="<nat>"):
    inputs = [f"{style} {s}" for s in sentences]
    encoded = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model.generate(**encoded, max_length=128)
    return [tokenizer.decode(g, skip_special_tokens=True) for g in outputs]

# Sample WMT test sentences
turkish_sentences = [...]
translations = translate_tr_sentences(turkish_sentences)
for i, (tr, en) in enumerate(zip(turkish_sentences, translations)):
    print(f"{i+1:>2}. TR: {tr}")
    print(f"    EN: {en}")

# Prepare tagged test file for batch eval
raw_test_src = Path("test.tr").read_text(encoding="utf-8").splitlines()
tagged_test_src = [f"<nat> {line.strip()}" for line in raw_test_src]
Path("test_tagged.tr").write_text("\n".join(tagged_test_src), encoding="utf-8")

# Translate test set
def generate_translations(inputs, batch_size=16):
    all_outputs = []
    for i in range(0, len(inputs), batch_size):
        batch = inputs[i:i+batch_size]
        encoded = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128)
        with torch.no_grad():
            outputs = model.generate(**encoded, max_length=128)
        decoded = [tokenizer.decode(t, skip_special_tokens=True) for t in outputs]
        all_outputs.extend(decoded)
    return all_outputs

# Evaluate predictions
pred_lines = generate_translations(tagged_test_src)
Path("test_pred.en").write_text("\n".join(pred_lines), encoding="utf-8")

# Calculate BLEU, chrF++, and BERTScore
from bert_score import score as bert_score
from sacrebleu import corpus_bleu, corpus_chrf
import pandas as pd

ref_lines = Path("test.en").read_text(encoding="utf-8").splitlines()
bleu = corpus_bleu(pred_lines, [ref_lines])
chrf = corpus_chrf(pred_lines, [ref_lines])
P, R, F1 = bert_score(pred_lines, ref_lines, lang="en", rescale_with_baseline=True)

scores_txt = (
    f"BLEU Score : {bleu.score:.2f}\n"
    f"chrF Score : {chrf.score:.2f}\n"
    f"BERTScore (F1) Avg : {F1.mean().item():.4f}\n"
)
print("Evaluation Metrics:\n" + scores_txt)

# Save evaluation results
output_dir = Path("/content/drive/MyDrive/NMT_LMs/Evaluation/")
output_dir.mkdir(parents=True, exist_ok=True)
df = pd.DataFrame({
    "source_tr": raw_test_src,
    "reference_en": ref_lines,
    "prediction_en": pred_lines,
    "bert_f1": F1.tolist()
})
df.to_csv(output_dir / "translation_results.csv", index=False, encoding="utf-8")
Path(output_dir / "scores.txt").write_text(scores_txt, encoding="utf-8")

print(f"All results saved to {output_dir}")


In [ ]:
# Load the baseline Turkish-English translation model and tokenizer from Hugging Face
baseline_model_name = "Helsinki-NLP/opus-mt-tr-en"
baseline_model = MarianMTModel.from_pretrained(baseline_model_name)
baseline_tokenizer = MarianTokenizer.from_pretrained(baseline_model_name)

# Mount Google Drive to access saved models
from google.colab import drive
drive.mount('/content/drive')

# Define paths to the fine-tuned and experimental model directories stored in Drive
FINETUNED_PATH    = "/content/drive/MyDrive/fine-tuned-opus-mt-tr-en"               # Fine-tuned on WMT data
NATURAL_PATH      = "/content/drive/MyDrive/NMT_LMs/Natural_NMT_Model"             # Fine-tuned with <nat>/<trans> tags
LINGUISTIC_PATH   = "/content/drive/MyDrive/morph-opus-mt2-tr-en"                   # Fine-tuned with morphological tags (ConcatMorph)

# Load all tokenizers and models into memory and set them to evaluation mode (inference only)

# Baseline model and tokenizer (no fine-tuning)
baseline_tok = MarianTokenizer.from_pretrained(baseline_model_name)
baseline_model = MarianMTModel.from_pretrained(baseline_model_name).eval()

# Fine-tuned model on WMT parallel data
finetuned_tok = MarianTokenizer.from_pretrained(FINETUNED_PATH)
finetuned_model = MarianMTModel.from_pretrained(FINETUNED_PATH).eval()

# Fine-tuned model trained on natural vs. translationese-tagged data
natural_tok = MarianTokenizer.from_pretrained(NATURAL_PATH)
natural_model = MarianMTModel.from_pretrained(NATURAL_PATH).eval()

# Morphologically-aware fine-tuned model (ConcatMorph input format)
ling_tok = MarianTokenizer.from_pretrained(LINGUISTIC_PATH)
ling_model = MarianMTModel.from_pretrained(LINGUISTIC_PATH).eval()


In [ ]:
# Define a helper function to translate a sentence using a given tokenizer and model
def translate(sentence, tokenizer, model):
    # Tokenize the input sentence and convert to tensor format
    inputs = tokenizer([sentence], return_tensors="pt", padding=True, truncation=True, max_length=128)

    # Disable gradient calculation (inference mode)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=128)

    # Decode the model output into a human-readable string
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


# Example Turkish sentence to be translated
example_tr = "Ege tavla oynamayı beceremiyor ama Özge hiç beceremiyor."
print("Source Sentence:", example_tr)

# Translate using the baseline pre-trained model (no fine-tuning)
print("Baseline MT Translation:")
print(translate(example_tr, baseline_tok, baseline_model))

# Translate using the WMT fine-tuned model
print("\nFinetuned MT Translation:")
print(translate(example_tr, finetuned_tok, finetuned_model))

# Translate using NaturalMT model with <nat> and <trans> style tags
print("\nNaturalMT Translation:")
print(translate(f"<nat> {example_tr}", natural_tok, natural_model))
print(translate(f"<trans>{example_tr}", natural_tok, natural_model))

# Translate a morphologically annotated sentence using the MorphMT model
print("\nMorphMT Translation (ConcatMorph format):")
print(translate(
    "Turkiyenin_ekonomiNoun+A3sg+Pnon+Nom son_Adv yılNoun+A3pl+P3sg+Loc büyük_Adj değişiklikNoun+A3pl+P3sg+Acc geçirVerb+Caus+Pos+Past+A3sg+.",
    ling_tok, ling_model
))